In [1]:
import pandas
import numpy as np


In [3]:
files_dir = "C:\\Users\\Francesco Marrocco\\Desktop\\WPI\\SENIOR\\DS3010\\Project\\restaurant_data"

In [ ]:
# open csv files and store them in a pandas dataset?

# read the csv files
train_full = pandas.read_csv(files_dir + "\\train_full.csv")

# print the first 5 rows of the dataset
print(train_full.head())

# print the df into a nice chart
print(train_full.describe())





C:\Users\Francesco Marrocco\AppData\Local\Temp\ipykernel_59796\1576698436.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_full = pandas.read_csv(files_dir + "\\train_full.csv")


  customer_id gender  status_x  verified_x         created_at_x  \
0     TCHWPBT   Male         1           1  2018-02-07 19:16:23   
1     TCHWPBT   Male         1           1  2018-02-07 19:16:23   
2     TCHWPBT   Male         1           1  2018-02-07 19:16:23   
3     TCHWPBT   Male         1           1  2018-02-07 19:16:23   
4     TCHWPBT   Male         1           1  2018-02-07 19:16:23   

          updated_at_x  location_number location_type  latitude_x  \
0  2018-02-07 19:16:23                0          Work      -96.44   
1  2018-02-07 19:16:23                0          Work      -96.44   
2  2018-02-07 19:16:23                0          Work      -96.44   
3  2018-02-07 19:16:23                0          Work      -96.44   
4  2018-02-07 19:16:23                0          Work      -96.44   

   longitude_x  ...  country_id  city_id         created_at_y  \
0        -67.2  ...         1.0      1.0  2018-01-30 14:42:04   
1        -67.2  ...         1.0      1.0  2018-05-03